# Named Entity Recognition without labelled data: A weak supervision approach

In this walkthrough, we will develop a neural NER model without access to labelled data. Let's look at a particular example of text (from Reuters):


In [1]:
import sys
sys.path.insert(0, '../..')


In [2]:
import re
news_text  = """
ATLANTA  (Reuters) - Retailer Best Buy Co, seeking new ways to appeal to cost-conscious shoppers, said on Tuesday it is selling refurbished 
 versions of Apple Inc's iPhone 3G at its stores that are priced about $50 less than new iPhones. 
 The electronics chain said the used iPhones, which were returned within 30 days of purchase, are priced at $149 for the model with 8 gigabytes of storage, 
 while the 16-gigabyte version is $249. A two-year service contract with AT&T Inc is required. New iPhone 3Gs currently sell for $199 and $299 at 
 Best Buy Mobile stores. "This is focusing on customers' needs, trying to provide as wide a range of products and networks for our consumers," said 
 Scott Moore, vice president of marketing for Best Buy Mobile. Buyers of first-generation iPhones can also upgrade to the faster refurbished 3G models at 
 Best Buy, he said. Moore said AT&T, the exclusive wireless provider for the iPhone, offers refurbished iPhones online. The sale of used iPhones comes as 
 Best Buy, the top consumer electronics chain, seeks ways to fend off increased competition from discounters such as Wal-Mart Stores Inc, which began 
 selling the popular phone late last month. Wal-Mart sells a new 8-gigabyte iPhone 3G for $197 and $297 for the 16-gigabyte model. The iPhone is also 
 sold at Apple stores and AT&T stores. Moore said Best Buy's move was not in response to other retailers' actions. (Reporting by  Karen Jacobs ; Editing 
 by  Andre Grenon )"""

news_text = re.sub('\\s+', ' ', news_text)

Let's view what the standard Spacy model produces (note it takes a few seconds to reload the vocabulary):

In [5]:
import spacy, skweak

# We load the spacy model
nlp = spacy.load("en_core_web_sm")
doc = nlp(news_text)

# Visualising the entities
skweak.utils.display_entities(doc)

The medium-size model works better, but still contains quite a few errors and omissions:

In [6]:

# We load the spacy model (takes a few seconds)
nlp = spacy.load("en_core_web_md")
doc = nlp(news_text)

# Visualising the entities
skweak.utils.display_entities(doc)

Ideally, one would wish to train a better named entity recognition model, which is better tailored to the specific needs and linguistic patterns found in those articles. However, although we have large amounts of raw text data, we often do not have text data labelled with named entities for our domain. We therefore worked on an alternative approach based on __weak supervision__, combining several (noisy) supervision sources instead of a single "gold standard". 

Indeed, we do have access to several possible supervision sources, such as alternative NER models trained on other corpora, large lists of entity (companies, person names, geographical locations), shallow linguistic patterns, and document-level constraints. 

The key idea behind the proposed approach is thus to (1) use these supervision sources to automatically annotate news corpora, (2) estimate a label model (more precisely an HMM model) that unifies all these sources into a single one, and (3) learn a new NER model based on these unified labels. <br>

__Outline of this notebook__: I describe below the various annotation schemes that I developed.  I then explain how these various sources can be merged into a single source using the `skweak` framework. Finally, I detail the architecture behind the NER model.

## __Step 1:__ Annotations

### 1) Annotators from other Spacy models

A first source of automatic annotation comes from NER models trained on multiple, distinct corpora. I went through [available NE-labelled corpora](https://github.com/juand-r/entity-recognition-datasets) to search for datasets that could be used to train alternative models. I then trained Spacy models for all of them, and then conducted some experiments to assess their performance. At the end of the process, I ended up with four models:
- The standard Spacy model for English (`en_core_web_md`), trained on Ontonotes v5
- A model trained on [ConLL 2003](https://www.clips.uantwerpen.be/conll2003/ner/)
- A model trained on the [Broad Twitter Corpus](https://github.com/GateNLP/broad_twitter_corpus)
- A model trained on a corpus of [SEC filings](https://www.aclweb.org/anthology/U15-1010/).
    
Note there are differences between the entity labels of these models: while Ontonotes contains no less than [18 classes](https://spacy.io/api/annotation#named-entities), the other corpora only contain `PER(SON)`, `ORG`, `LOC` and `MISC`. Furthermore, the labels also do not match each other perfectly: while Ontonotes distinguishes between geopolitical locations (`GPE`) and "natural" locations (such as continents, seas etc., labelled as `LOC`), the three other models regroup all geographical entities as `LOC`. 

We can apply annotations from a Spacy model using the `ModelAnnotator` class

In [7]:
annotator = skweak.spacy_model.ModelAnnotator("conll2003", "../data/conll2003/")

doc = annotator(doc)
skweak.utils.display_entities(doc, "conll2003")

As we can see, the results are not perfect on this model either, but the errors are distinct from the ones made by the Ontonotes model. 

The annotations are written in the `user_data` dictionary of the Spacy document:

In [8]:
doc.user_data["spans"]["conll2003"]

{(1, 2): 'LOC',
 (3, 4): 'ORG',
 (7, 10): 'ORG',
 (31, 33): 'ORG',
 (97, 99): 'ORG',
 (102, 104): 'ORG',
 (114, 117): 'ORG',
 (145, 147): 'PER',
 (153, 156): 'ORG',
 (174, 176): 'ORG',
 (180, 181): 'PER',
 (204, 206): 'ORG',
 (224, 229): 'ORG',
 (240, 243): 'ORG',
 (267, 268): 'ORG',
 (273, 274): 'PER',
 (275, 277): 'ORG',
 (292, 294): 'PER',
 (297, 299): 'PER'}

Each `ModelAnnotator` adds two annotation sources: one that is directly based on the Spacy Model (here `conll2003`), and one that also includes the corrections specified in the method `_correct_entities` (in `spacy_wrapper.py`) that we implemented earlier this year.  The corrected version are indicated with a `+c` suffix.

Here are the results from the three other models:

In [10]:
core_web_annotator = skweak.spacy_model.ModelAnnotator("core_web_md", "en_core_web_md")

doc = core_web_annotator(doc)
skweak.utils.display_entities(doc, "core_web_md")


__Note__: When annotating large collections of news documents, the method `annotator.pipe(news_docs)` is much more efficient than calling `annotate(...)` every single time, as it batches the documents on which to run the NER model.

### 2) Annotators from gazetteers

Another useful source of annotation comes from large lists of entities such as persons, places and organisations. The gazetteers are using a _trie_ to efficiently search for occurrences in the text. Each gazetteer creates two annotation sources: one that is case-sensitive (`_cased` suffix) and one case-insentitive (`_uncased` suffix).


#### 2.1) Wikipedia
The database from Wikipedia is extracted from the [NECKar](https://event.ifi.uni-heidelberg.de/?page_id=532) dataset.  The postprocessing (which, among others, filters out entities that are also relatively common English words) is implemented in `compile_wikidata` as [WIKIDATA](https://github.com/NorskRegnesentral/skweak/releases/download/v0.2/wikidata_tokenised.json.gz). In addition, I also extracted from Wikidata a list of commercial products and added them to the gazetteer. 

In [13]:
import domains.ner.conll2003_ner

tries = skweak.gazetteers.extract_json_data("../data/wikidata_tokenised.json")
annotator = skweak.gazetteers.GazetteerAnnotators("wiki", tries)

annotator(doc)
skweak.utils.display_entities(doc, "wiki_*")

Extracting data from /mn/sarpanitu/ansatte-u2/aliaksah/PycharmProjects/skweak3/data/wikidata_tokenised.json
Populating trie for class PERSON (number: 2621131)
Populating trie for class LOC (number: 47104)
Populating trie for class GPE (number: 601419)
Populating trie for class ORG (number: 295449)
Populating trie for class PRODUCT (number: 12457)


Again, the annotation model does make some errors: `Moore` is thought to be a [geopolitical entity](https://en.wikipedia.org/wiki/Moore) instead of a person. Note that `AT&T` has two alternative labels: `ORG` or `GPE` (see [AT&T station](https://en.wikipedia.org/wiki/AT%26T_(SEPTA_station))). The data is available as [WIKI_SMALL](https://github.com/NorskRegnesentral/skweak/releases/download/v0.2/wikidata_small_tokenised.json.gz).

In addition to the full wiki data, I also added a specific gazetteer that only employs wikidata objects containing a text description:

In [17]:
tries = skweak.gazetteers.extract_json_data("../data/wikidata_small_tokenised.json")
annotator = skweak.gazetteers.GazetteerAnnotators("wikismall", tries)

annotator(doc)
skweak.utils.display_entities(doc, "wikismall_*")

Extracting data from /mn/sarpanitu/ansatte-u2/aliaksah/PycharmProjects/skweak3/data/wikidata_small_tokenised.json
Populating trie for class PERSON (number: 1863434)
Populating trie for class LOC (number: 14241)
Populating trie for class GPE (number: 273373)
Populating trie for class ORG (number: 91341)
Populating trie for class PRODUCT (number: 12457)


As we can see, the "cased" gazetteers have a higher precision than the uncased gazetteers (at a cost of lower coverage).

#### 2.2 Crunchbase

The second gazetteer [Crunchbase](https://github.com/NorskRegnesentral/skweak/releases/download/v0.2/crunchbase.json.gz) is extracted from the [Open Data Map from Crunchbase](https://data.crunchbase.com/docs/open-data-map), which contains lists of both organisations and (business) persons.

In [18]:
tries = skweak.gazetteers.extract_json_data("../data/crunchbase.json",  spacy_model="en_core_web_sm")
annotator = skweak.gazetteers.GazetteerAnnotators("crunchbase", tries)

annotator(doc)
skweak.utils.display_entities(doc, "crunchbase_*")

Extracting data from /mn/sarpanitu/ansatte-u2/aliaksah/PycharmProjects/skweak3/data/crunchbase.json
Populating trie for class COMPANY (number: 788714)
Populating trie for class ORG (number: 261)
Populating trie for class PERSON (number: 1062669)


#### 2.3 Geonames

The [geonames](http:www.geonames.org) database [GEO_NAMES](https://github.com/NorskRegnesentral/skweak/blob/main/data/geonames.json) contains a large list of locations, including both geopolitical entities and "natural" locations:

In [19]:
tries = skweak.gazetteers.extract_json_data("../data/geonames.json",  spacy_model="en_core_web_sm")
annotator = skweak.gazetteers.GazetteerAnnotators("geo", tries)

annotator(doc)
skweak.utils.display_entities(doc, "geo_*")

Extracting data from /mn/sarpanitu/ansatte-u2/aliaksah/PycharmProjects/skweak3/data/geonames.json
Populating trie for class GPE (number: 15205)


#### 2.4 Product names

Finally, I used [DBPedia](http://www.dbpedia.org) to extract a list of products and brands as [Products](https://github.com/NorskRegnesentral/skweak/blob/main/data/products.json), since the recognition of products is particularly poor in Spacy NER models:

In [20]:

tries = skweak.gazetteers.extract_json_data("../data/products.json",  spacy_model="en_core_web_sm")
annotator = skweak.gazetteers.GazetteerAnnotators("products", tries)

annotator(doc)
skweak.utils.display_entities(doc, "products_*")

Extracting data from /mn/sarpanitu/ansatte-u2/aliaksah/PycharmProjects/skweak3/data/products.json
Populating trie for class PRODUCT (number: 45362)


### 3. Shallow patterns

Some named entities can also be captured through relatively simple, handcrafted patterns defined on the Spacy document. The class `FunctionAnnotator` makes it easy to define an annotator based on a function that takes a Spacy document as input and generate text spans with a label. Relations of mutual exclusivity between annotation sources can also be specified in the annotator. For instance, we can specify that numbers that are part of a date, time or money span should be ignored from the "number_detector" (to avoid having e.g. the `21` in `October 21` labelled as a `CARDINAL`): 

In [21]:

date_annotator = skweak.heuristics.FunctionAnnotator("date_detector", domains.ner.conll2003_ner.date_generator)
time_annotator = skweak.heuristics.FunctionAnnotator("time_detector", domains.ner.conll2003_ner.time_generator)
money_annotator = skweak.heuristics.FunctionAnnotator("money_detector", domains.ner.conll2003_ner.money_generator)
exclusives = ["date_detector", "time_detector", "money_detector"]
number_annotator = skweak.heuristics.FunctionAnnotator("number_detector", domains.ner.conll2003_ner.number_generator)
number_annotator.add_incompatible_sources(exclusives)

date_annotator(doc)
time_annotator(doc)
money_annotator(doc)
number_annotator(doc)
skweak.utils.display_entities(doc, "date_detector")
skweak.utils.display_entities(doc, "time_detector")
skweak.utils.display_entities(doc, "money_detector")
skweak.utils.display_entities(doc, "number_detector")

I have also created a range of patterns aiming to improve the _detection_ of named entities, even though they leave the actual label underspecified (as a generic `ENT` label). Four such detectors are constructed:
- two detectors of proper names based on casing (marking sequence of tokens whose lemma are "titled" as potential named entities)
- one detector of NNP sequences (based on the Spacy POS tagger)
- and one detector of sequences with proper names linked with "compound" dependency relations

In [22]:
# Detection based on casing
proper_detector = skweak.heuristics.TokenConstraintAnnotator("proper_detector", skweak.utils.is_likely_proper, "ENT")
    
# Detection based on casing, but allowing some lowercased tokens
proper2_detector = skweak.heuristics.TokenConstraintAnnotator("proper2_detector", skweak.utils.is_likely_proper, "ENT")
proper2_detector.add_gap_tokens(domains.ner.conll2003_ner.LOWERCASED_TOKENS | domains.ner.conll2003_ner.NAME_PREFIXES)
#add  .ner.        
# Detection based on part-of-speech tags
nnp_detector = skweak.heuristics.TokenConstraintAnnotator("nnp_detector", lambda tok: tok.tag_=="NNP", "ENT")
        
# Detection based on dependency relations (compound phrases)
compound = lambda tok: skweak.utils.is_likely_proper(tok) and skweak.utils.in_compound(tok)
compound_detector = skweak.heuristics.TokenConstraintAnnotator("compound_detector", compound, "ENT")
 
combined = skweak.base.CombinedAnnotator()
exclusives = ["date_detector", "time_detector", "money_detector"]
for annotator in [proper_detector, proper2_detector, nnp_detector, compound_detector]:
    annotator.add_incompatible_sources(exclusives)
    annotator.add_gap_tokens(["'s", "-"])
    combined.add_annotator(annotator)

    # We add one variants for each NE detector, looking at infrequent tokens
    infrequent_name = "infrequent_%s"%annotator.name
    combined.add_annotator(skweak.heuristics.SpanConstraintAnnotator(infrequent_name, annotator.name, skweak.utils.is_infrequent))

doc = combined(doc)
skweak.utils.display_entities(doc, "proper_detector")
skweak.utils.display_entities(doc, "proper2_detector")
skweak.utils.display_entities(doc, "nnp_detector")
skweak.utils.display_entities(doc, "compound_detector")

Finally, I created three specific annotators:
- to recognise company names with a legal type
- full person names (with a first name along a list of common first names)
- slightly less common entities such as `NORP`, `FAC`, `LANGUAGE`, `EVENT` and `LAW`.

In [23]:

# Other types (legal references etc.)      
misc_detector = skweak.heuristics.FunctionAnnotator("misc_detector", domains.ner.conll2003_ner.misc_generator)
legal_detector = skweak.heuristics.FunctionAnnotator("legal_detector", domains.ner.conll2003_ner.legal_generator)
        
# Detection of companies with a legal type
ends_with_legal_suffix = lambda x: x[-1].lower_.rstrip(".") in domains.ner.conll2003_ner.LEGAL_SUFFIXES
company_type_detector = skweak.heuristics.SpanConstraintAnnotator("company_type_detector", "proper2_detector", 
                                                    ends_with_legal_suffix, "COMPANY")
# Detection of full person names
FIRSR_NAMES = "../data/first_names.json"
full_name_detector = skweak.heuristics.SpanConstraintAnnotator("full_name_detector", "proper2_detector", 
                                                     domains.ner.conll2003_ner.FullNameDetector(), "PERSON")


legal_detector(doc)
company_type_detector(doc)
full_name_detector(doc)
misc_detector(doc)
skweak.utils.display_entities(doc, "company_type_detector")
skweak.utils.display_entities(doc, "full_name_detector")
skweak.utils.display_entities(doc, "misc_detector")
skweak.utils.display_entities(doc, "legal_detector")

Finally, we also rely on an external probabilistic [parser of named entities](https://github.com/snipsco/snips-nlu-parsers) from [Snips](https://snips.ai/). The parser recognises `DATE`, `TIME`, `ORDINAL`, `CARDINAL`, `MONEY` and `PERCENT`. The parser is implemented in _Rust_, so it runs quite fast.

In [24]:
# Detection based on a probabilistic parser
snips = domains.ner.conll2003_ner.SnipsAnnotator("snips")
snips(doc)
skweak.utils.display_entities(doc, "snips")

### 4. Document-level annotators

All annotators presented so far rely on _local_ decisions on tokens or phrases.  However, news articles are not mere collections of words, but exhibit a high degree of internal coherence. This can be exploited to furhter improve the annotation. Two document-level annotators are implemented:

Before we can run the document-level annotators, we need to normalise some of the entities. The `StandardiseAnnotator` is responsible for this normalisation:
- entities `PER` (from conll2003, BTC and SEC) are set to `PERSON`
- entities `LOC` from conll2003, BTC and SEC for spans that are also annotated by other layers as `GPE` are set to `GPE` 
- entities `ORG` that are annotated by other layers as `COMPANY` are set to `COMPANY`
    

In [25]:
annotator = domains.ner.conll2003_ner.ConLL2003Standardiser()
doc = annotator(doc)

In [26]:
mv = skweak.aggregation.MajorityVoter("mv", ["LOC", "MISC", "ORG", "PER"])
mv.add_underspecified_label("ENT", {"LOC", "MISC", "ORG", "PER"})
doc = mv(doc)
skweak.utils.display_entities(doc, "mv")

#### 4.1 Document history

When a journalist first mentions an entity such as a company or person in an article, they typically write it in a "long form", and then use shorter mentions once the entity is properly introduced. For instance, in the text above, "Scott Moore" is first mentioned with a full name, and then simply referred to as "Moore". Similarly, companies are often first introduced to with their legal type.  The `DocumentHistoryAnnotator` takes advantage of this property, by propagating the label from the first mention onto subsequent mentions:

In [27]:
annotator = skweak.doclevel.DocumentHistoryAnnotator("doc_history", "mv", ["PER", "ORG"])
annotator(doc)
skweak.utils.display_entities(doc, "doc_history_org_cased")

#### 4.2 Label consistency

Another property of news documents is the fact that two (or more) named entities sharing the same string in a text typically refer to the same entity, and should therefore have the same label. "Komatsu" can be both a company name and a city in Japan, but within a given document, it will typically be one or the other for the whole document. We can capture this fact with an annotator that looks at the majority label for a given string, and annotate all occurrences with this label:

In [28]:
annotator = skweak.doclevel.DocumentMajorityAnnotator("doc_majority", "mv")
annotator(doc)
skweak.utils.display_entities(doc, "doc_majority_org_cased")

In [29]:
mv = skweak.aggregation.MajorityVoter("mv", ["LOC", "MISC", "ORG", "PER"])
mv.add_underspecified_label("ENT", {"LOC", "MISC", "ORG", "PER"})
doc = mv(doc)
skweak.utils.display_entities(doc, "mv")

## __Step 2__: Estimation of label model

We can construct a full annotator with all annotators described above, and then run it on a dataset such as Reuters, Bloomberg, or Acquire:

In [30]:
#full_annotator = domains.conll2003_ner.NERAnnotator().add_shallow().add_doc_level()
full_annotator = domains.ner.conll2003_ner.NERAnnotator().add_all()
print("Total number of annotators:", len(full_annotator.annotators))

Loading shallow functions
Loading Spacy NER models
Loading gazetteer supervision modules
Extracting data from /mn/sarpanitu/ansatte-u2/aliaksah/PycharmProjects/skweak3/data/wikidata_tokenised.json
Populating trie for class PERSON (number: 2621131)
Populating trie for class LOC (number: 47104)
Populating trie for class GPE (number: 601419)
Populating trie for class ORG (number: 295449)
Populating trie for class PRODUCT (number: 12457)
Extracting data from /mn/sarpanitu/ansatte-u2/aliaksah/PycharmProjects/skweak3/data/wikidata_small_tokenised.json
Populating trie for class PERSON (number: 1863434)
Populating trie for class LOC (number: 14241)
Populating trie for class GPE (number: 273373)
Populating trie for class ORG (number: 91341)
Populating trie for class PRODUCT (number: 12457)
Extracting data from /mn/sarpanitu/ansatte-u2/aliaksah/PycharmProjects/skweak3/data/geonames.json
Populating trie for class GPE (number: 15205)
Extracting data from /mn/sarpanitu/ansatte-u2/aliaksah/PycharmPr

We can then take the raw data from [reuters](https://github.com/NorskRegnesentral/skweak/releases/download/v0.2/reuters_small.docbin), run Spacy on the textual content, and finally apply the annotator to get annotations from the each source:

In [32]:
# We annotate 200 documents, and store them in a Spacy DocBin file
full_annotator.annotate_docbin("../data/reuters_small.docbin", cutoff=200)
#full_annotator.annotate_docbin("./data/reuters_small.docbin", "./data/reuters_small.docbin")

Number of processed documents: 1
Number of processed documents: 2
Number of processed documents: 3
Number of processed documents: 4
Number of processed documents: 5
Number of processed documents: 6
Number of processed documents: 7
Number of processed documents: 8
Number of processed documents: 9
Number of processed documents: 10
Number of processed documents: 11
Number of processed documents: 12
Number of processed documents: 13
Number of processed documents: 14
Number of processed documents: 15
Number of processed documents: 16
Number of processed documents: 17
Number of processed documents: 18
Number of processed documents: 19
Number of processed documents: 20
Number of processed documents: 21
Number of processed documents: 22
Number of processed documents: 23
Number of processed documents: 24
Number of processed documents: 25
Number of processed documents: 26
Number of processed documents: 27
Number of processed documents: 28
Number of processed documents: 29
Number of processed doc

Once this is done, we can finally estimate a unified annotator model through weak supervision. The basic idea is to describe the named entity recognition problem as a _Hidden markov Model_ where the observations are the annotations from each source, and the states correspond to the "true" (hidden) labels for each token, as illustrated in Figure 2 in the paper.

Since we don't have access to the true labels for each token, we will rely on _Baum-Welch_ (a variant of EM) to estimate the HMM model through unsupervised training. More specifically, we will need to estimate 3 models:
- the initial probabilities $P(Y_0)$ of the labels for the first token of a document
- the transition matrix $P(Y_i | Y_{i-1})$ for the labels 
- the emission models $P(\lambda_{i,j} | Y_i)$ of observing a particular value $\lambda_{i,j}$ (say, `B-PER`) from the source $j$ given the true label $Y_i$. In the current model, we assume the emissions to be independent of one another given the true label, to reduce the complexity of the model.

Given an annotated dataset, the HMM model can be easily estimated:

In [33]:
unified_model = skweak.aggregation.HMM("hmm", ["LOC", "MISC", "ORG", "PER"])
unified_model.add_underspecified_label("ENT", ["LOC", "MISC", "ORG", "PER"])
# We then run Baum-Welch on the model (can take some time)
unified_model.fit("../data/reuters_small.docbin")

# Saving the model to a file
unified_model.save("../data/hmm_reuters_small.pkl")

Starting iteration 1
Finished E-step with 200 documents
Starting iteration 2


         1     -256440.8198             +nan


Finished E-step with 200 documents
Starting iteration 3


         2     -242474.8003      +13966.0195


Finished E-step with 200 documents
Starting iteration 4


         3     -241999.7229        +475.0775


Finished E-step with 200 documents


         4     -241877.5856        +122.1373


Note that the HMM model relies on some informative priors to facilitate the parameter estimation:
- the prior for the initial probabilities is a Dirichlet based on counts for the most reliable model (chosen right now to be the Spacy NER model trained on Ontonotes)
- the prior for the transition matrix is a list of Dirichlet also based on counts from the standard Spacy NER model.
- finally, the initial emission models are calculated based on subjective estimates of the relative precision and recall of each source. For instance, we know that a source like `company_type_detector` (which looks at legal suffixes such as "Inc." at the end of the noun phrase) has a very high precision, but a low recall , since many mentions of companies do not include a suffix. In contrast, gazeteers will tend to have a better recall, but a lower precision (some company names also happen to be names of geopolitical entities or persons).  The initial precisions and recalls provided to the model are specified in `SOURCE_PRIORS` in the file `labelling.py`. When a precision and recall is not provided for a given source, they are assumed to be zeros (for instance, `company_type_detector` only detects `COMPANY` entities and nothing else).  

Once the model is learned, we can apply it as any other "annotator" object:

In [34]:
unified_model.pretty_print() 

HMM model on following sources: ['multitoken_wiki_small_product_uncased', 'multitoken_wiki_gpe_cased', 'wiki_org_uncased', 'multitoken_wiki_small_gpe_uncased', 'doc_history_org_uncased', 'companies_org_uncased', 'infrequent_proper_detector', 'multitoken_wiki_small_loc_uncased', 'companies_company_uncased', 'wiki_gpe_cased', 'multitoken_wiki_person_cased', 'full_name_detector', 'edited_BTC', 'wiki_small_gpe_cased', 'wiki_small_gpe_uncased', 'doc_majority_loc_cased', 'company_type_detector', 'multitoken_wiki_person_uncased', 'wiki_person_cased', 'multitoken_wiki_small_org_cased', 'core_web_md_truecase', 'multitoken_wiki_small_org_uncased', 'wiki_product_cased', 'wiki_small_person_cased', 'multitoken_crunchbase_company_uncased', 'doc_majority_per_uncased', 'crunchbase_company_cased', 'companies_company_cased', 'multitoken_products_product_cased', 'wiki_small_loc_uncased', 'doc_majority_org_cased', 'doc_majority_misc_cased', 'number_detector', 'doc_majority_org_uncased', 'wiki_small_produc

            O  B-LOC  I-LOC  B-MISC  I-MISC  B-ORG  I-ORG  B-PER  I-PER  B-ENT  I-ENT
O        1.00   0.00   0.00     0.0    0.00    0.0    0.0    0.0    0.0    0.0    0.0
B-LOC    0.96   0.04   0.00     0.0    0.00    0.0    0.0    0.0    0.0    0.0    0.0
I-LOC    0.94   0.00   0.06     0.0    0.00    0.0    0.0    0.0    0.0    0.0    0.0
B-MISC   1.00   0.00   0.00     0.0    0.00    0.0    0.0    0.0    0.0    0.0    0.0
I-MISC   0.99   0.00   0.00     0.0    0.01    0.0    0.0    0.0    0.0    0.0    0.0
B-ORG    1.00   0.00   0.00     0.0    0.00    0.0    0.0    0.0    0.0    0.0    0.0
I-ORG    1.00   0.00   0.00     0.0    0.00    0.0    0.0    0.0    0.0    0.0    0.0
B-PER    1.00   0.00   0.00     0.0    0.00    0.0    0.0    0.0    0.0    0.0    0.0
I-PER    1.00   0.00   0.00     0.0    0.00    0.0    0.0    0.0    0.0    0.0    0.0
weights  0.39   0.35   0.35     0.5    0.50    0.5    0.5    0.5    0.5    0.5    0.5
--------
Emission model for source: doc_majority_org_c

weights  0.34   0.67   0.35     0.5    0.50    0.5    0.5    0.5    0.5    0.5    0.5
--------
Emission model for source: core_web_md
            O  B-LOC  I-LOC  B-MISC  I-MISC  B-ORG  I-ORG  B-PER  I-PER  B-ENT  I-ENT
O        1.00   0.00   0.00    0.00    0.00   0.00   0.00   0.00   0.00   0.00   0.00
B-LOC    0.05   0.86   0.05    0.00    0.00   0.02   0.01   0.00   0.00   0.00   0.00
I-LOC    0.10   0.00   0.82    0.00    0.00   0.00   0.07   0.00   0.00   0.00   0.00
B-MISC   0.07   0.00   0.00    0.82    0.02   0.08   0.00   0.00   0.00   0.00   0.00
I-MISC   0.05   0.00   0.01    0.00    0.85   0.00   0.09   0.00   0.00   0.00   0.00
B-ORG    0.07   0.01   0.00    0.00    0.00   0.81   0.08   0.02   0.00   0.00   0.00
I-ORG    0.05   0.00   0.00    0.00    0.00   0.01   0.92   0.00   0.01   0.00   0.00
B-PER    0.05   0.00   0.00    0.00    0.00   0.11   0.00   0.82   0.00   0.00   0.00
I-PER    0.03   0.00   0.00    0.00    0.00   0.00   0.02   0.02   0.93   0.00   0.00
weight

Emission model for source: multitoken_crunchbase_org_uncased
            O  B-LOC  I-LOC  B-MISC  I-MISC  B-ORG  I-ORG  B-PER  I-PER  B-ENT  I-ENT
O        1.00    0.0    0.0     0.0    0.00    0.0    0.0    0.0    0.0    0.0    0.0
B-LOC    1.00    0.0    0.0     0.0    0.00    0.0    0.0    0.0    0.0    0.0    0.0
I-LOC    1.00    0.0    0.0     0.0    0.00    0.0    0.0    0.0    0.0    0.0    0.0
B-MISC   1.00    0.0    0.0     0.0    0.00    0.0    0.0    0.0    0.0    0.0    0.0
I-MISC   0.99    0.0    0.0     0.0    0.01    0.0    0.0    0.0    0.0    0.0    0.0
B-ORG    1.00    0.0    0.0     0.0    0.00    0.0    0.0    0.0    0.0    0.0    0.0
I-ORG    1.00    0.0    0.0     0.0    0.00    0.0    0.0    0.0    0.0    0.0    0.0
B-PER    1.00    0.0    0.0     0.0    0.00    0.0    0.0    0.0    0.0    0.0    0.0
I-PER    1.00    0.0    0.0     0.0    0.00    0.0    0.0    0.0    0.0    0.0    0.0
weights  0.34    0.5    0.5     0.5    0.50    0.5    0.5    0.5    0.5    0.5 

In [35]:
reader = skweak.utils.docbin_reader("../data/reuters_small.docbin")
doc = next(reader)
#full_annotator(doc)
unified_model(doc)
skweak.utils.display_entities(doc, "hmm")
doc2 = next(reader)
#full_annotator(doc2)
unified_model(doc2)
skweak.utils.display_entities(doc2, "hmm")

And we can apply it to the full dataset:

In [36]:
unified_model.annotate_docbin("../data/reuters_small.docbin")

Number of processed documents: 1
Number of processed documents: 2
Number of processed documents: 3
Number of processed documents: 4
Number of processed documents: 5
Number of processed documents: 6
Number of processed documents: 7
Number of processed documents: 8
Number of processed documents: 9
Number of processed documents: 10
Number of processed documents: 11
Number of processed documents: 12
Number of processed documents: 13
Number of processed documents: 14
Number of processed documents: 15
Number of processed documents: 16
Number of processed documents: 17
Number of processed documents: 18
Number of processed documents: 19
Number of processed documents: 20
Number of processed documents: 21
Number of processed documents: 22
Number of processed documents: 23
Number of processed documents: 24
Number of processed documents: 25
Number of processed documents: 26
Number of processed documents: 27
Number of processed documents: 28
Number of processed documents: 29
Number of processed doc

<br>

## __Step 3__: Development of neural NER model


We can now learn a neural NER model based on these unified annotations. We have two options: a straighforward (but slightly underperforming) approach using Spacy, and a more sophisticated approach using our own NER model

### __Alternative 1__: Using Spacy

In [37]:
skweak.utils.json_writer(skweak.utils.docbin_reader("../data/reuters_small.docbin", cutoff=20), "../data/reuters_small_dev.json", "hmm")
skweak.utils.json_writer(skweak.utils.docbin_reader("../data/reuters_small.docbin", nb_to_skip=20),  "../data/reuters_small_train.json", "hmm")


Writing JSON file to /mn/sarpanitu/ansatte-u2/aliaksah/PycharmProjects/skweak3/data/reuters_small_dev.json
Writing JSON file to /mn/sarpanitu/ansatte-u2/aliaksah/PycharmProjects/skweak3/data/reuters_small_train.json


And we can then directly train a new NER model with Spacy's training regime:

In [38]:
!rm -rf ../data/reuters_small_spacy
import spacy
spacy.cli.train(lang="en", output_path="../data/reuters_small_spacy", 
                train_path="../data/reuters_small_train.json", 
                dev_path="../data/reuters_small_dev.json",
               base_model="en_core_web_md", vectors="en_core_web_md",
               pipeline="ner", n_iter=10, replace_components=True)

✔ Created output directory:
/mn/sarpanitu/ansatte-u2/aliaksah/PycharmProjects/skweak3/data/reuters_small_spacy
Training pipeline: ['ner']
Starting with base model 'en_core_web_md'
Replacing component from base model 'ner'
Loading vector from model 'en_core_web_md'
Counting training words (limit=0)


  1%|▏         | 1184/88545 [00:00<00:12, 6863.39it/s]


Itn  NER Loss   NER P   NER R   NER F   Token %  CPU WPS
---  ---------  ------  ------  ------  -------  -------


  1%|          | 826/88545 [00:00<00:13, 6574.98it/s]  

  1  11715.082  64.561  64.760  64.660  100.000    29232


...

  1%|          | 907/88545 [00:00<00:11, 7882.79it/s]  

  8   1151.225  89.645  89.645  89.645  100.000    38916


  1%|▏         | 1149/88545 [00:00<00:11, 7730.90it/s] 

  9   1072.273  89.767  89.490  89.628  100.000    37177


 10    870.499  89.214  89.490  89.352  100.000    39415
✔ Saved model to output directory
/mn/sarpanitu/ansatte-u2/aliaksah/PycharmProjects/skweak3/data/reuters_small_spacy/model-final
eating best model...✔ Created best model
/mn/sarpanitu/ansatte-u2/aliaksah/PycharmProjects/skweak3/data/reuters_small_spacy/model-best


We then load the learned model (taking the model version with best performance on the development set):

In [39]:
nlp = spacy.load("../data/reuters_small_spacy/model-best")

And we finally run it on the document:

In [40]:
skweak.utils.display_entities(nlp(news_text))

# __STEP 3__: Evaluation

## Evaluation on ConLL 2003

We evaluate the approach on a standard dataset, namely [conLL 2003](https://github.com/NorskRegnesentral/skweak/blob/main/data/conll2003.docbin). Note that ConLL only makes use of 4 labels (`PER`, `ORG`, `LOC`, `MISC`) instead of the 18 labels which were used above.


And we can then apply the NER model to annotate CoNLL 2003:

In [41]:
unified_model.annotate_docbin("../data/conll2003.docbin", "../data/conll2003.docbin")

Number of processed documents: 1
Number of processed documents: 2
Number of processed documents: 3
Number of processed documents: 4
Number of processed documents: 5
Number of processed documents: 6
Number of processed documents: 7
Number of processed documents: 8
Number of processed documents: 9
Number of processed documents: 10
Number of processed documents: 11
Number of processed documents: 12
Number of processed documents: 13
Number of processed documents: 14
Number of processed documents: 15
Number of processed documents: 16
Number of processed documents: 17
Number of processed documents: 18
Number of processed documents: 19
Number of processed documents: 20
Number of processed documents: 21
Number of processed documents: 22
Number of processed documents: 23
Number of processed documents: 24
Number of processed documents: 25
Number of processed documents: 26
Number of processed documents: 27
Number of processed documents: 28
Number of processed documents: 29
Number of processed doc

Number of processed documents: 248
Number of processed documents: 249
Number of processed documents: 250
Number of processed documents: 251
Number of processed documents: 252
Number of processed documents: 253
Number of processed documents: 254
Number of processed documents: 255
Number of processed documents: 256
Number of processed documents: 257
Number of processed documents: 258
Number of processed documents: 259
Number of processed documents: 260
Number of processed documents: 261
Number of processed documents: 262
Number of processed documents: 263
Number of processed documents: 264
Number of processed documents: 265
Number of processed documents: 266
Number of processed documents: 267
Number of processed documents: 268
Number of processed documents: 269
Number of processed documents: 270
Number of processed documents: 271
Number of processed documents: 272
Number of processed documents: 273
Number of processed documents: 274
Number of processed documents: 275
Number of processed 

Number of processed documents: 483
Number of processed documents: 484
Number of processed documents: 485
Number of processed documents: 486
Number of processed documents: 487
Number of processed documents: 488
Number of processed documents: 489
Number of processed documents: 490
Number of processed documents: 491
Number of processed documents: 492
Number of processed documents: 493
Number of processed documents: 494
Number of processed documents: 495
Number of processed documents: 496
Number of processed documents: 497
Number of processed documents: 498
Number of processed documents: 499
Number of processed documents: 500
Number of processed documents: 501
Number of processed documents: 502
Number of processed documents: 503
Number of processed documents: 504
Number of processed documents: 505
Number of processed documents: 506
Number of processed documents: 507
Number of processed documents: 508
Number of processed documents: 509
Number of processed documents: 510
Number of processed 

Number of processed documents: 724
Number of processed documents: 725
Number of processed documents: 726
Number of processed documents: 727
Number of processed documents: 728
Number of processed documents: 729
Number of processed documents: 730
Number of processed documents: 731
Number of processed documents: 732
Number of processed documents: 733
Number of processed documents: 734
Number of processed documents: 735
Number of processed documents: 736
Number of processed documents: 737
Number of processed documents: 738
Number of processed documents: 739
Number of processed documents: 740
Number of processed documents: 741
Number of processed documents: 742
Number of processed documents: 743
Number of processed documents: 744
Number of processed documents: 745
Number of processed documents: 746
Number of processed documents: 747
Number of processed documents: 748
Number of processed documents: 749
Number of processed documents: 750
Number of processed documents: 751
Number of processed 

Number of processed documents: 962
Number of processed documents: 963
Number of processed documents: 964
Number of processed documents: 965
Number of processed documents: 966
Number of processed documents: 967
Number of processed documents: 968
Number of processed documents: 969
Number of processed documents: 970
Number of processed documents: 971
Number of processed documents: 972
Number of processed documents: 973
Number of processed documents: 974
Number of processed documents: 975
Number of processed documents: 976
Number of processed documents: 977
Number of processed documents: 978
Number of processed documents: 979
Number of processed documents: 980
Number of processed documents: 981
Number of processed documents: 982
Number of processed documents: 983
Number of processed documents: 984
Number of processed documents: 985
Number of processed documents: 986
Number of processed documents: 987
Number of processed documents: 988
Number of processed documents: 989
Number of processed 

The file `eval_utils.py` contains code to easily extract evaluation metrics by comparing the annotations from a particular annotation layer (for instance the HMM predictions, or the predictions from a single source) to the gold standard: